Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [257]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq
from icecream import ic

In [258]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [259]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def is_solvable(state:np.ndarray)->bool:
    inv_count = 0
    tmp_state = state.copy().reshape(PUZZLE_DIM**2)
    for i in range(0,PUZZLE_DIM**2):
        for j in range(i+1,PUZZLE_DIM**2):
            if tmp_state[j] != 0 and tmp_state[i] != 0 and tmp_state[i] > tmp_state[j]:
                inv_count += 1
    return (inv_count%2) == 0 

In [260]:
RANDOMIZE_STEPS = 100
target_state=np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
start_state = target_state
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    start_state = do_action(start_state, choice(available_actions(start_state)))


Randomizing:   0%|          | 0/100 [00:00<?, ?it/s]

In [261]:
class Position: #The position class represent the state of the game with the f (h+g) embedded.
    def __init__(self,grid:np.array,g,prev_pos=None) -> None:
        self.grid=grid
        self.prev_pos=prev_pos
        self.f=self.__h()+g
    def __h(self):
        #This is the heuristic: MANHATTAN DISTANCE over every element
        tot_dist=0
        for i in range(1,PUZZLE_DIM**2):
            x_1,y_1=np.where(target_state==i)
            x_2,y_2=np.where(self.grid==i)
            tot_dist+=abs(x_1-x_2)+abs(y_1-y_2)
        return tot_dist
    
    # #Overridng to use comparisons
    # def __eq__(self,other):
    #    return self.grid==other.grid
    
    #Overriding to use heap
    def __lt__(self, other):
     return self.f<other.f
    
    




In [262]:
evaluated_pos=0#evaluated position (cost of the algorithm)

def a_star(start_pos:Position):
    global evaluated_pos
    current_cost=0 #depth (cost of g)
    curr_pos=start_pos
    closed_list = set() #Visited Positions
    open_list=[] #Visitable Positions

    while open_list or current_cost==0:
        closed_list.add(tuple(map(tuple, curr_pos.grid))) #hashable
        if((curr_pos.grid==target_state).all()):
            ic(len(closed_list))
            ic(len(open_list))
            return curr_pos
        evaluated_pos+=1
        new_actions=available_actions(curr_pos.grid)
        for action in new_actions:
            new_pos=Position(do_action(curr_pos.grid,action),current_cost+1,curr_pos)
            if tuple(map(tuple, new_pos.grid)) not in closed_list:
                heapq.heappush(open_list, new_pos)
        current_cost+=1
        curr_pos=heapq.heappop(open_list)
        if(evaluated_pos%1000==0):
            ic(curr_pos.grid)

ic(is_solvable(start_state))
ic("Start pos:")
ic(start_pos.grid,start_pos.f)
start_pos=Position(start_state,0)
found_sol=a_star(start_pos)   


ic("Positions evaluated:",evaluated_pos)



ic("Steps:")
while True:
    ic(found_sol.grid)
    if(found_sol.prev_pos==None):
        break
    found_sol=found_sol.prev_pos

ic| is_solvable(start_state): True
ic| 'Start pos:'
ic| start_pos.grid: array([[3, 6, 4],
                           [5, 7, 1],
                           [2, 8, 0]])
    start_pos.f: array([16])
ic| curr_pos.grid: array([[2, 3, 4],
                          [7, 1, 6],
                          [0, 8, 5]])
ic| curr_pos.grid: array([[1, 2, 4],
                          [3, 7, 5],
                          [8, 6, 0]])
ic| curr_pos.grid: array([[4, 6, 8],
                          [2, 7, 0],
                          [5, 1, 3]])
ic| curr_pos.grid: array([[8, 0, 1],
                          [4, 2, 6],
                          [7, 5, 3]])
ic| curr_pos.grid: array([[8, 4, 1],
                          [2, 7, 6],
                          [5, 3, 0]])
ic| curr_pos.grid: array([[2, 8, 6],
                          [0, 5, 1],
                          [4, 7, 3]])
ic| curr_pos.grid: array([[0, 8, 4],
                          [6, 5, 3],
                          [2, 1, 7]])
ic| curr_pos.grid: a